[Node 65: Verteilte Programme mit XML-RPC](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node65.html)

Navigation:

**Next:** [Aufgaben](node66.ipynb) **Up:** [Netzwerkprogrammierung](node62.ipynb) **Previous:** [Module der Python-Standardbibliothek](node64.ipynb)

##  Verteilte Programme mit XML-RPC

XML-RPC (XML = Extensible Markup Language; RPC = Remote Procedure Call) ist ein synchrones Protokoll zum Aufruf entfernter Methoden: Es wird eine Methode in einem anderen Prozess oder Rechner aufgerufen und muss auf die Antwort gewartet werden. Der Aufruf kann mehrere Parameter haben und ein Ergebnis. 

Argumente und Rückgabewerte mit Typen werden als XML übertragen, ein Protokoll, das unabhängig von XML-RPC besteht, genauso wie das Übertragungsprotokoll HTTP oder HTTPS. (XML haben wir hier in [XML Basics](node55.ipynb) eingeführt.) 

Die Verwendung existierender Protokolle in XML-RPC erleichtert die Implementierung. XML-RPC ist ein etablierter, spachunabhängiger Standard, was erlaubt, dass Klienten und Server in unterschiedlichen Sprachen programmiert sein können. Eine Weiterentwicklung von XML-RPC ist z.B. der Industriestandard SOAP (Simple Object Access Protocol).

In Python kann das [``xmlrpc``](https://docs.python.org/3/library/xmlrpc.html)-Model verwendet werden, das Server- und Klientfunktionalität bereitstellt.

Wir betrachten zwei Beispielprogramme:
* [``xmlrpc_server.py``](source/xmlrpc_server.py) demonstriert, wie man einen XML-Server definiert und startet.  
* [``xmlrpc_client.py``](source/xmlrpc_client.py) demonstriert, wie ein Klient die Server-Methoden aufruft. 

Der Server verwendet die Klasse `SimpleXMLRPCServer`:

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
SimpleXMLRPCServer?

Bei Ausführung beider Programme gibt der Server ähnlich einem Webserver Daten über eingehende Abfragen aus:

In [ ]:
!python3 source/xmlrpc_server.py
# run "python3 xmlrpc_client.py" in external shell

Um die XML-Nachrichten des Klienten zu sehen, setzen wir in `xmlrpc_client.py` das Flag ``verbose=True``:
```python
prx = xmlrpclib.ServerProxy('http://localhost:7070',
                            allow_none=True, verbose=True)
```
Man beachte im Code, dass wir uns nicht um eine Kapselung der Daten und Datentypen kümmern brauchen. Dies wird vom Standard durch die Bereitstellung einer Reihe von grundlegenden Datentypen übernommen (Integer, Gleitkommazahl, boolesche Variablen, Zeichenketten, ein eigene Typ für Datum mit Uhrzeit und base64-kodierte binäre Daten). Diese können zu verschachtelten Listen und Sammlungen von Schlüssel-Wert-Paaren kombiniert werden.

---

Ein Nachteil des bis jetzt konfigurierten XML-RPC-Servers ist, dass er nur einen Thread verwendet. 
Nachteile daraus sind:  
* Funktion benötigt viel Zeit oder blockiert 
* Verbindung zwischen Klient und Server ist besonders langsam 
<!-- * Klient nimmt Daten nur sehr langsam entgegen  -->
<!-- * viele Daten werden zwischen Klient und Server gesendet  ??-->

Der Server sollte also mit mehreren Threads gestaltet werden:
* [``xmlrpc_threadedserver.py``](source/xmlrpc_threadedserver.py) demonstiert, wie solch ein multi-threaded XML-Server definiert und gestartet wird.
(Dies verwendet das Konzept der ["mix-in"s](https://coderbook.com/@marcus/deep-dive-into-python-mixins-and-multiple-inheritance/).)

In [ ]:
!python3 source/xmlrpc_threadedserver.py

Zur Demonstration wird in einem Fenster eine schnelle Methode wiederkehrend aufgerufen:
```python
import time
import xmlrpc.client 
prx = xmlrpc.client.ServerProxy('http://localhost:7070') 
while True: 
    print(prx.FastFunc()) 
    time.sleep(0.1) 
```

Gleichzeitig wird in einem anderen Fenster eine langsame Methode aufgerufen:
```python
import xmlrpc.client
prx = xmlrpc.client.ServerProxy('http://localhost:7070')
prx.SlowFunc()
```